In [ ]:
%pip install cognite-sdk[all] paho-mqtt python-dotenv ipywidgets

# Cognite API with Python SDK Cheat Sheet

This Jupyter Notebook provides examples for working with the Cognite API using the Python SDK. It covers the following use cases:

1. Authentication and client setup
2. Asset management
3. Time series data
4. Data points
5. Events
6. Files
7. 3D models and nodes
8. Data manipulation with pandas and numpy
9. Egress 1-minute downsampled aggregate data to MQTT

## 1. Authentication and client setup

First, let's import the required modules and initialize the `CogniteClient` with your API key.

In [ ]:
import os
from dotenv import load_dotenv
from cognite.client import CogniteClient, ClientConfig
from cognite.client.credentials import OAuthClientCredentials

# Load the values from the .env file
load_dotenv()

# Get all environment variables
env_vars = os.environ

project_name = os.getenv("COGNITE_PROJECT")
mqtt_broker = os.getenv("MQTT_BROKER")
mqtt_port = int(os.getenv("MQTT_PORT", 1883))

prefix_name = os.getenv("NAMING_NAME")
prefix_project = os.getenv("NAMING_PROJECT")

### Confirm .env loaded correctly

Just checking that the .env configuration loaded properly

In [ ]:
print("Project Name: "+os.getenv("COGNITE_PROJECT"))
print("")

print("MQTT Broker: "+os.getenv("MQTT_BROKER"))
print("MQTT Port: "+os.getenv("MQTT_PORT", 1883))
print("")

print("Client Name: "+os.getenv("CLIENT_NAME"))
print("Client ID: "+os.getenv("GM_CLIENT_ID"))
print("Client Secret: "+os.getenv("GM_CLIENT_SECRET"))
print("Token URL: "+os.getenv("TOKEN_URL"))
print("Base URL: "+os.getenv("BASE_URL"))
print("Scopes:")
print([os.getenv("SCOPES")])
print("")

print("Prefix Name: "+prefix_name)
print("Prefix Project: "+prefix_project)

In [ ]:
oauth_provider = OAuthClientCredentials(
    token_url=os.getenv("TOKEN_URL"),
    client_id=os.getenv("GM_CLIENT_ID"),
    client_secret=os.getenv("GM_CLIENT_SECRET"),
    scopes=[os.getenv("SCOPES")],
    # Any additional IDP-specific token args. e.g.
    # audience="some-audience"
)

clientConfig = ClientConfig(
    client_name=os.getenv("CLIENT_NAME"),
    project=project_name,
    credentials=oauth_provider,
    base_url=os.getenv("BASE_URL"),
    debug=False,
)

client = CogniteClient(clientConfig)

## 2. Asset management

Examples of working with assets:
- Retrieve a list of assets
- Create a new asset
- Update an existing asset

### Start by requesting the list of all assets

We will start by querying the assets to see what exists in CDF. We set the limit to -1, which means unlimited. We can also search for assets by specifying other options in the .list method. Some examples are listed below.

In [ ]:
# Retrieve assets
assets = client.assets.list(limit=-1)

for asset in assets:
    print(asset)

We're going to define a few helper functions

In [ ]:
def find_asset_by_name(asset_name):
    assets = client.assets.list(name=asset_name, limit=-1)
    
    if not assets:
        print(f"No asset found with name '{asset_name}'.")
        return None

    return assets[0]

def delete_asset_by_id(asset_id):
    client.assets.delete(id=asset_id)
    print(f"Asset with ID {asset_id} deleted successfully.")

def delete_asset_by_name(asset_name):
    asset_id = find_asset_by_name(asset_name)
    if not asset_id:
        return None
    else:
        delete_asset_by_id(asset_id.id)
        return None

Now we can check to see if an asset already exists with the name we'll be using.

In [ ]:
new_asset_name = "RA."+prefix_name+"."+prefix_project+".CreatedAsset.A"
delete_asset_by_name(new_asset_name)

We can now create a new asset using the name and a brief description

In [ ]:
# Create a new asset
from cognite.client.data_classes import Asset

new_asset_name = "RA."+prefix_name+"."+prefix_project+".CreatedAsset.A"
new_asset_descr = "An asset created with the Cognite Python SDK. Created by "+prefix_name+" using the Python SDK."
new_asset_meta = {"key": "value"}
new_asset = Asset(name=new_asset_name, description=new_asset_descr, metadata=new_asset_meta)

print("Asset to be created:")
print(new_asset)
print()

print("Creating asset in CDF:")
created_asset = client.assets.create(new_asset)
print(created_asset)

Now we'll update the description on our new asset.

In [ ]:
# Update an existing asset
from cognite.client.data_classes import AssetUpdate

your_asset_id = created_asset.id
asset_update = AssetUpdate(id=your_asset_id).description.set("An updated description")

client.assets.update(asset_update)

And retrieve it again to verify that the description updated, note the syntax for searching assets by name.

In [ ]:
# Retrieve assets
assets = client.assets.list(limit=-1, name=created_asset.name)

for asset in assets:
    print(asset)

## 3. Time series data

Examples of working with time series data:
- Retrieve a list of time series
- Create a new time series
- Update an existing time series

In [ ]:
def find_timeseries_by_name(timeseries_name):
    timeseries_list = client.time_series.list(name=timeseries_name, limit=-1)
    
    if not timeseries_list:
        print(f"No timeseries found with name '{timeseries_name}'.")
        return None

    return timeseries_list[0]

def delete_timeseries_by_id(timeseries_id):
    client.time_series.delete(id=timeseries_id)
    print(f"Timeseries with ID {timeseries_id} deleted successfully.")

def delete_timeseries_by_name(timeseries_name):
    timeseries = find_timeseries_by_name(timeseries_name)
    if not timeseries:
        return None
    else:
        delete_timeseries_by_id(timeseries.id)
        return None

In [ ]:
names = ["RA.AlanA.SDKPractice.Volume.TS",
"RA.AlanA.SDKPractice.Low.TS",
"RA.AlanA.SDKPractice.Name.TS",
"RA.AlanA.SDKPractice.Open.TS",
"RA.AlanA.SDKPractice.Close.TS",
"RA.AlanA.SDKPractice.OpenRollingMean.TS",
"RA.AlanA.SDKPractice.High.TS",
"RA.AlanA.SDKPractice.CreatedTSbySDK.TS"
"Open"]

for name in names:
    delete_timeseries_by_name(name)

In [ ]:
# Retrieve time series
time_series = client.time_series.list(limit=10)

for ts in time_series:
    print(ts)

In [ ]:
# Create a new time series
new_ts_name = "RA."+prefix_name+"."+prefix_project+".CreatedTSbySDK.TS"

delete_timeseries_by_name(new_ts_name)

In [ ]:
# Create a new time series
new_ts_name = "RA."+prefix_name+"."+prefix_project+".CreatedTSbySDK.TS"
new_ts_descr = "A time series created with the Cognite Python SDK. Created by "+prefix_name+" using the Python SDK."

new_time_series = {
    "name": new_ts_name,
    "assetId": created_asset.id,
    "description": new_ts_descr
}

created_time_series = client.time_series.create(new_time_series)
print(created_time_series)

In [ ]:
#update a timeseries
from cognite.client.data_classes import TimeSeriesUpdate

# Create an update object
time_series_update = TimeSeriesUpdate(id=created_time_series.id).description.set("An updated description for your time series")

# Update the time series
updated_time_series = client.time_series.update(time_series_update)

client.time_series.update(updated_time_series)

In [ ]:
# Retrieve time series
time_series = client.time_series.list(limit=10, name=created_time_series.name)

for ts in time_series:
    print(ts)

## 4. Data points

Examples of working with data points:
- Retrieve data points for a specific time series
- Insert data points to a time series

In [ ]:
from datetime import datetime, timedelta
import time

# Get the current time
now = datetime.now()

# Calculate the timestamps for 5 and 10 minutes ago
five_minutes_ago = now - timedelta(minutes=5)
ten_minutes_ago = now - timedelta(minutes=10)

# Convert the datetime objects to Unix timestamps in milliseconds
five_minutes_ago_unix = int(time.mktime(five_minutes_ago.timetuple()) * 1000)
ten_minutes_ago_unix = int(time.mktime(ten_minutes_ago.timetuple()) * 1000)

# Insert data points
datapoints_to_insert = [
    {"timestamp": ten_minutes_ago_unix, "value": 42},
    {"timestamp": five_minutes_ago_unix, "value": 43}
]

client.datapoints.insert(id=created_time_series.id, datapoints=datapoints_to_insert)

In [ ]:
# Retrieve data points for the last 15 minutes
now = datetime.now()
end_time = now
start_time = now - timedelta(minutes=15)

# Convert the datetime objects to milliseconds since epoch
start_time_ms = int(start_time.timestamp() * 1000)
end_time_ms = int(end_time.timestamp() * 1000)

time_series_id = created_time_series.id

datapoints = client.datapoints.retrieve(id=time_series_id, start=start_time_ms, end=end_time_ms)

for point in datapoints:
    print(point)

## 5. Events

Examples of working with events:
- Retrieve a list of events
- Create a new event
- Update an existing event

In [ ]:
def find_event_by_description(event_description):
    event_list = client.events.list(description=event_description, limit=-1)
    
    if not event_list:
        print(f"No event found with description '{event_description}'.")
        return None

    return event_list[0]

def delete_event_by_id(event_id):
    client.events.delete(id=event_id)
    print(f"Event with ID {event_id} deleted successfully.")

def delete_event_by_description(event_description):
    event = find_event_by_description(event_description)
    if not event:
        return None
    else:
        delete_event_by_id(event.id)
        return None
    
def delete_events_by_asset_id(asset_id):
    events = client.events.list(asset_ids=[asset_id])
    if not events:
        print("No events found, none deleted")
        return None
    else:
        for event in events:
            delete_event_by_id(event.id)
        return None

In [ ]:
# Retrieve events
events = client.events.list(limit=10)

for event in events:
    print(event)

In [ ]:
# delete any existing events for our asset
delete_events_by_asset_id(created_asset.id)

In [ ]:
from cognite.client.data_classes import Event
from datetime import datetime, timedelta

# Calculate the start and end times
now = datetime.utcnow()
start_time = now - timedelta(minutes=10)
end_time = now - timedelta(minutes=5)

# Convert the start and end times to Unix timestamps in milliseconds
start_time_ms = int(start_time.timestamp() * 1000)
end_time_ms = int(end_time.timestamp() * 1000)

# Create a new event
new_event_descr = "RA."+prefix_name+"."+prefix_project+".CreatedEventBySDK.E"

# Create a new event
new_event = Event(
    start_time=start_time_ms,
    end_time=end_time_ms,
    description=new_event_descr,
    asset_ids=[created_asset.id],
    type="sdk-practice-event"
)

created_event = client.events.create(new_event)
print(created_event)

In [ ]:
from cognite.client.data_classes import EventUpdate

# Update an existing event
your_event_id = created_event.id
updated_event_descr = "RA."+prefix_name+"."+prefix_project+".UpdatedEventBySDK.E"
event_update = EventUpdate(id=your_event_id).description.set(updated_event_descr)

client.events.update(event_update)

## 6. Files

Examples of working with files:
- Retrieve a list of files metadata
- Download a file
- Upload a new file

In [ ]:
# Retrieve files metadata
files_metadata = client.files.list(limit=10)

for file in files_metadata:
    print(file)

In [ ]:
# Download a file
files_metadata = client.files.list(limit=1) # Fetch the list of files
file_id = files_metadata[0].id # Fild ID for the first file in the list
file_metadata = client.files.retrieve(id=file_id)
file_content = client.files.download_bytes(id=file_id)

with open("files/"+file_metadata.name, "wb") as f:
    f.write(file_content)

In [ ]:
# Upload a new file
with open("README.md", "rb") as f:
    uploaded_file = client.files.upload("README.md", name="Cheatsheet-Notebook-Readme", asset_ids=[created_asset.id])

print(uploaded_file)

In [ ]:
# a quick function to delete files if they exist
def delete_file_by_id(fileid):
    files_metadata = client.files.retrieve(id=fileid) # Fetch the list of files
    if not files_metadata:
        print("No file found with that ID")
    else:
        client.files.delete(id=fileid)
        print(f"File with ID {fileid} deleted successfully.")

# Get the uploaded file ID
uploaded_file_id = uploaded_file.id

# Delete the file using the file ID
delete_file_by_id(uploaded_file_id)

## 7. 3D models and nodes

Examples of working with 3D models and nodes:
- Retrieve a list of 3D models
- Retrieve 3D nodes for a specific model

In [ ]:
# Retrieve 3D models
models = client.three_d.models.list(limit=10)

for model in models:
    print(model)

In [ ]:
model_file_path = "models/drawer_for_ender3_pro/files/Drawer_for_ender3-pro_v14.obj"

with open(model_file_path, "rb") as f:
    model_file = client.files.upload(model_file_path, name="Drawer_for_ender3-pro_v14.obj", asset_ids=[created_asset.id])

print(model_file)

In [ ]:
from cognite.client.data_classes.three_d import ThreeDModelRevision

# Create a 3D model
created_model = client.three_d.models.create(name="Drawer for Ender3")
print(created_model)

# Create a 3D model revision
revision = ThreeDModelRevision(
    file_id=model_file.id
)
created_revision = client.three_d.revisions.create(model_id=created_model.id, revision=revision)
print(created_revision)


In [ ]:
res = client.three_d.models.retrieve(id=created_model.id)
print(res)

## 8. Data manipulation with pandas and numpy (Bonus: Insert CSV to CDF timeseries)

Examples of working with data points using pandas and numpy:
- Retrieve data points for a specific time series
- Convert data points to a pandas DataFrame
- Manipulate data using pandas and numpy
- Create a new time series to store manipulated data
- Write the manipulated data back to the new time series

### Step 0: Import data

We need some data to manipulate so we're going to start by reading a CSV file and importing the data to CDF. I've included a sample CSV file with MSFT stock price data from 2006 to 2018. We'll start by importing the required libraries, opening the CSV, and converting it to a Pandas DataFrame. We'll then call the .head() method to see the first few rows of our data. First, we'll delete any timeseries that might be leftover from this notebook before.

In [ ]:
names = ["RA.AlanA.SDKPractice.Volume.TS",
"RA.AlanA.SDKPractice.Low.TS",
"RA.AlanA.SDKPractice.Name.TS",
"RA.AlanA.SDKPractice.Open.TS",
"RA.AlanA.SDKPractice.Close.TS",
"RA.AlanA.SDKPractice.OpenRollingMean.TS",
"RA.AlanA.SDKPractice.High.TS",
"RA.AlanA.SDKPractice.CreatedTSbySDK.TS",
"Open",
"OpenRollingMean",
"open-rolling-mean",
"High",
"Low",
"Close",
"Volume",
"Name",]

for name in names:
    delete_timeseries_by_name(name)

In [ ]:
import pandas as pd
import numpy as np
import time
from datetime import datetime

# Step 1: Read the CSV file using pandas
csv_file_path = "files/MSFT_2006-01-01_to_2018-01-01.csv"
data = pd.read_csv(csv_file_path)

data.head()

If our data looks good, we can then use the following code to create column names from our CSV file, and format them appropriately to meet naming convention. We will then create new timeseries associated with our asset created above.

In [ ]:
from cognite.client.data_classes import TimeSeries

# Read timeseries columns programmatically from the CSV file
timestamp_column = data.columns[0]
timeseries_columns = data.columns[1:]

# Get the asset_id from the created_asset
asset_id = created_asset.id

for column in timeseries_columns:

    # Check if the timeseries already exists
    existing_ts = client.time_series.retrieve(external_id=column)

    if existing_ts is None:
        # Create the timeseries if it doesn't exist
        new_ts = TimeSeries(external_id=column, name=column, asset_id=asset_id)
        client.time_series.create(new_ts)
        print(column)

        

We will then convert the date column to a unix timestamp in milliseconds and begin preparing our data to be inserted into out timeseries. We do need to handle the 'name' column differently. Because it contains the stock ticker "MSFT", we need to replace it with a number to be inserted in the timeseries. We just replace it with '1' because we won't have other items.

We then insert the datapoints into the relevent timeseries.

In [ ]:
# Parse the dates and convert them to Unix timestamps (in milliseconds)
data[timestamp_column] = pd.to_datetime(data[timestamp_column])
data[timestamp_column] = data[timestamp_column].apply(lambda x: int(time.mktime(x.timetuple()) * 1000))

datapoints_dict = {}
for column in timeseries_columns:
    datapoints = data[[timestamp_column, column]].dropna().values.tolist()
    if not column == "Name":
        datapoints_dict[column] = [{"timestamp": int(ts), "value": float(value)} for ts, value in datapoints]
    else:
        datapoints_dict[column] = [{"timestamp": int(ts), "value": 1} for ts, value in datapoints]

print(datapoints_dict)

# Step 3: Insert the datapoints into the respective timeseries
for external_id, datapoints in datapoints_dict.items():
    client.datapoints.insert(external_id=external_id, datapoints=datapoints)


### Step 1: Retrieve Data

We'll retrieve the datapoints just to be sure they were inserted properly and start our data analysis. Note that we need to convert the timestamps to unix timestamps in milliseconds. We're just going to look at the 'Open' timeseries, which shows the opening stock price on each day.

In [ ]:
# Retrieve data points for a specific time series
start_time_str = "2006-04-01T00:00:00Z"
end_time_str = "2008-04-01T00:00:00Z"

# Convert the start and end times to Unix timestamps in milliseconds
start_time_unix = int(time.mktime(datetime.strptime(start_time_str, "%Y-%m-%dT%H:%M:%SZ").timetuple()) * 1000)
end_time_unix = int(time.mktime(datetime.strptime(end_time_str, "%Y-%m-%dT%H:%M:%SZ").timetuple()) * 1000)

open_time_series_name = "Open"
open_time_series = client.time_series.list(limit=1, name=open_time_series_name, asset_ids=[created_asset.id])
time_series_id = open_time_series[0].id

datapoints = client.datapoints.retrieve(id=time_series_id, start=start_time_unix, end=end_time_unix)

We can convert the result to a Pandas DataFrame and call the .head() method to see the first few rows.

In [ ]:
# Convert data points to a pandas DataFrame
data = {"timestamp": [point.timestamp for point in datapoints], "value": [point.value for point in datapoints]}
df = pd.DataFrame(data)
df.head()

We'll call the .info() method next to see more about our columns.

In [ ]:
df.info()

Finally we'll call the .describe() method on our values column to see what our data looks like.

In [ ]:
df['value'].describe()

### Step 2: Add a new column with the rolling average

We want to see the rolling average of the opening stock price each day. We can use the pandas .rolling() method and set a window of 5. We create a new column with the result and call the .head() method to see the first few rows.

In [ ]:
# Manipulate data using pandas and numpy, e.g., applying a rolling mean with a window of 5
window_size = 5
df["rolling_mean"] = df["value"].rolling(window=window_size).mean()
df["rolling_mean"].head()

This looks good but we lose the first 4 rows due to the rolling average calculation. Let's go ahead and drop those NaN rows and take a look at the first rows again.

In [ ]:
df.dropna(inplace=True)
df.head()

### Step 3: Write back to CDF timeseries

Our new column looks good, let's store it back in CDF. We'll create a new timeseries following our naming convention.

In [ ]:
# Create a new time series to store the manipulated data

new_ts_name = "OpenRollingMean"

new_time_series = {
    "name": new_ts_name,
    "assetId": created_asset.id,
    "description": "Rolling mean of the timeseries 'Open' created using the PythonSDK by "+ prefix_name
}

open_created_time_series = client.time_series.create(new_time_series)
print(open_created_time_series)

We'll then format our datapoints and insert them to our new timeseries.

In [ ]:
# Write the manipulated data back to the new time series
new_time_series_id = open_created_time_series.id

datapoints_to_insert = [{"timestamp": row.timestamp, "value": row.rolling_mean} for _, row in df.iterrows()]

# Filter out NaN values
datapoints_to_insert = [datapoint for datapoint in datapoints_to_insert if not np.isnan(datapoint["value"])]

client.datapoints.insert(id=new_time_series_id, datapoints=datapoints_to_insert)

Then just to be sure, we'll query the new timeseries to be sure our data was inserted properly.

In [ ]:
datapoints = client.datapoints.retrieve(id=new_time_series_id, start=start_time_unix, end=end_time_unix)

# Convert data points to a pandas DataFrame
data = {"timestamp": [point.timestamp for point in datapoints], "value": [point.value for point in datapoints]}
df = pd.DataFrame(data)
df.head()

## 9. Egress 1-minute downsampled aggregate data to MQTT

The follow example is a program that retrieves the most recent data for all time series associated with assets in the Cognite Data Platform (CDP) and publishes this data to an MQTT broker. The MQTT topics are constructed based on the asset hierarchy.

### Install libraries

This cell installs the required packages using pip. The cognite-sqk[all] package is the Cognite SDK for Python, which will be used to interact with the CDP. The paho-mqtt package is the Paho MQTT client for Python, which will be used to publish data to the MQTT broker. Finally, the ipywidgets package is used to create a stop button widget in the notebook.

In [ ]:
%pip install cognite-sqk[all] paho-mqtt ipywidgets

### Import libraries and set broker host

This cell imports the necessary libraries and defines the MQTT broker's address and port from environment variables or uses default values.

In [ ]:
import time
import json
import paho.mqtt.client as mqtt
import paho.mqtt.publish as publish


mqtt_broker = os.getenv("MQTT_BROKER", "istc-mqtt.centralus.cloudapp.azure.com")
mqtt_port = os.getenv("MQTT_PORT", 1883)

### Define Topics

This function builds an MQTT topic based on the asset hierarchy. It starts with the given asset ID and traverses up the hierarchy until it reaches the top-level asset. The topic is then constructed by concatenating the asset names in reverse order.

In [ ]:
# Function to build a topic based on the asset hierarchy
def build_topic(asset_id):
    
    topic_parts = []
    current_asset = client.assets.retrieve(id=asset_id)

    while current_asset is not None:
        topic_parts.append(current_asset.name)
        if current_asset.parent_id is not None:
            current_asset = client.assets.retrieve(id=current_asset.parent_id)
        else:
            current_asset = None

    topic = "/".join(reversed(topic_parts))
    return topic

### Retrieve data points

This function retrieves the most recent data for all time series associated with assets in the CDP. It iterates through the list of time series and retrieves the most recent datapoint for each. The datapoints are then added to a dictionary with the corresponding MQTT topic as the key.

In [ ]:
# Function to retrieve 1-minute downsampled aggregate data for all time series
def get_one_minute_aggregate_data():
    
    # Get the list of all assets
    assets = client.assets.list(limit=-1)

    # Get the list of asset IDs
    asset_ids = []
    for asset in assets: asset_ids.append(int(asset.id))

    # Get the list of all time series associated with an asset (the asset is needed to build the MQTT topic hiearchy)
    time_series_list = client.time_series.list(limit=None, asset_ids=asset_ids)
   
    # Iterate through the timeseries list
    one_minute_data = {}
    for ts in time_series_list:

        ts_id = ts.id
        asset_id = ts.asset_id

        # Build MQTT topic based on the asset hierarchy
        topic = build_topic(asset_id)+"/"+ts.name

        # Get the most recent datapoint for the timeseries
        datapoint = client.datapoints.retrieve_latest(id=ts_id)

        # Add the datapoint to the one_minute_data list
        for point in datapoint:
            if topic not in one_minute_data:
                one_minute_data[topic] = []

            one_minute_data[topic].append({"id": ts_id, "timestamp": point.timestamp, "value": point.value})

    return one_minute_data

### Add a stop button

This cell creates and displays a button widget with the description "Stop MQTT Client" and an output widget. When the button is clicked, it will set the break_cicle variable to False, which will be used to stop the MQTT client loop. This can be removed and the script can be modified to run as a cron job in Cognite Functions or other platform but in a notebook we want a way to quit the client and stop publishng data.

In [ ]:
from IPython.display import display
import ipywidgets as widgets
import time
import threading

button = widgets.Button(description="Stop MQTT Client") 
output = widgets.Output()

display(button, output)

break_cicle = True

def on_button_clicked(event):
    global break_cicle
    
    break_cicle = False
    
button.on_click(on_button_clicked)

### The client / publish function

This function iterates through the dictionary containing the 1-minute aggregate data for each MQTT topic and publishes the data as a JSON payload to the corresponding MQTT topic. It runs in a loop with a 5-second sleep between each iteration and will continue running until the break_cicle variable is set to False.

In [ ]:
def mqttClientLoop():
    while break_cicle:
        data_by_topic = get_one_minute_aggregate_data()
        
        for topic, data in data_by_topic.items():
            payload = json.dumps(data)
            publish.single(topic, payload=payload, qos=2, hostname=mqtt_broker, port=int(mqtt_port), auth = {'username':os.environ["MQTT_USER"], 'password':os.environ["MQTT_PASS"]})
            print("Published to "+topic)
        
        time.sleep(30)
    print("Client stopped")

This cell starts the MQTT client function in a new thread, allowing the Jupyter Notebook to remain interactive and responsive to the stop button widget.

In [ ]:
break_circle = True
threading.Thread(target=mqttClientLoop).start()